## Inputs

In [ ]:
#Path to strength test reports
folder = r"C:\Users\remaa\Documents\[Project-Personal]\2017-2098 (Powell River)\Concrete Strength Test"

## Calculations

### Imports

In [ ]:
import pdfplumber, re, os, openpyxl, subprocess
from openpyxl.styles import Font, Alignment, Border, Side
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from fpdf import FPDF
from itertools import zip_longest
pd.options.mode.chained_assignment = None  # default='warn'

files = [os.path.join(folder, file) for file in os.listdir(folder) if file.lower().endswith('.pdf')]

#### Classes

In [ ]:
class concrete_test():
    def __init__(self, file):
        self.file = file
        with pdfplumber.open(file) as pdf:
            data = []
            for i in range(len(pdf.pages)):
                page = pdf.pages[i]
                data.append(page.extract_text())
            self.data = "\n".join(data)
            

        self.find_company()

        self.test_data_cols = ['Specimen','Cure','Test_Date', 'Age', 'Compressive_Str']
        self.extracted_data = {
            "Filename": os.path.basename(file),
            "Filepath": file,
            "Company": self.company,
            "Report Date": None,
            "Test Data": pd.DataFrame(columns=self.test_data_cols),
            "Set Num": None,
            "Specimens": 0,
            "Cast Date": None,
            "Transported Date": None,
            "Specified Strength": None,
            "Specified Strength Days": None,
            "Admixtures": [],
            "Mix Number": None,
            "Load Vol": None,
            "Slump": None,
            "Specified Slump": [],
            "Air": None,
            "Specified Air": [],
            "Location Comments": [],
            "Other Comments": [],
            "Report Date": None,
            "Errors": pd.DataFrame(columns=['Error Code', 'Description'])
        }

        if self.company:
            if self.company == "McElhanney":
                self.McElhanney()
            if self.company == "Kontur Geotechnical Consultants":
                self.Kontur()
            self.check_errors()
        else:
            self.log_error('a',self.data.split('\n')[0].strip())

    def combine_sheets(self, combine):
        for key in combine.keys():
            if self.extracted_data['Mix Number'] in combine[key]:
                self.extracted_data['Mix Number'] = key

    def find_company(self):
        company = None
        if 'mcelhanney' in self.data.split('\n')[0].lower():
            company = "McElhanney"
        if 'kontur' in self.data.split('\n')[0].lower():
            company = "Kontur Geotechnical Consultants"
        self.company = company
    
    def log_error(self, code, desc):
        df = pd.DataFrame([[code, desc]],columns = ['Error Code', 'Description'])
        self.extracted_data['Errors'] = pd.concat([self.extracted_data['Errors'], df], ignore_index=True)

    def McElhanney(self):
        try:
            pattern = {
                "Test Data": r".*([A-Z]) *Cylinder *(Lab|Field) *(\d*) *((\d{2}-\w{3})-? *(\d*) *[A-Za-z ]*)?\d{3}.\d *\d{3}.\d( *\d* *(\d*.\d))?",
                "Specified Strength": r"SPECIFIED STRENGTH\s*(\d*)MPa *@ *(\d*) DAYS",
                "Set Data": r".*SET *NO.(\d*) *SPECIMENS *(\d*) *CAST *(\d{2}-\w{3}-\d{4}) *TRANSPORTED *(\d{2}-\w{3}-\d{4})",
                "Report Date": r".*Page *1 *of *\d* *(\d{4}.\w{3}.\d*)|(\d*-\w{3}-\d{4})",
                "Air": r".*AIR *(\d{1,2}.?\d?) *% *SPEC. *(\d{1,2}.?\d?) *± *(\d{1,2}.?\d?)",
                "Slump": r".*SLUMP *(\d*) *mm *SPEC. *(\d*) *± *(\d*)",
                "Mix": r".*MIX *NO. *(.*)$",
                "Load Volume": r"^LOAD *VOL. *(\d*) m3"
            }
            for i, line in enumerate(self.data.split('\n')):
                specified_strength = re.match(pattern['Specified Strength'], line.strip())
                if specified_strength:
                    self.extracted_data["Specified Strength"] = int(specified_strength.group(1))
                    self.extracted_data["Specified Strength Days"] = int(specified_strength.group(2))

                set_data = re.match(pattern["Set Data"], line)
                if set_data:
                    self.extracted_data["Set Num"] = set_data.group(1)
                    self.extracted_data["Specimens"] = int(set_data.group(2))
                    self.extracted_data["Cast Date"] = datetime.strptime(set_data.group(3),"%d-%b-%Y")
                    self.extracted_data["Transported Date"] = datetime.strptime(set_data.group(4),"%d-%b-%Y")
                    continue

                test_data = re.match(pattern["Test Data"], line)
                if test_data:
                    data = [test_data.group(1), test_data.group(2), test_data.group(5), test_data.group(6), test_data.group(8)]
                    if data[2]:
                        if datetime.strptime(data[2] + "-" +str(datetime.now().year), "%d-%b-%Y") > datetime.now():
                            data[2] = datetime.strptime(data[2] + "-" +str(datetime.now().year - 1), "%d-%b-%Y")
                        else:
                            data[2] = datetime.strptime(data[2] + "-" +str(datetime.now().year), "%d-%b-%Y")
                    df = pd.DataFrame([data],columns = self.test_data_cols)
                    self.extracted_data['Test Data'] = pd.concat([self.extracted_data['Test Data'], df], ignore_index=True)
                    continue
                
                report_date = re.match(pattern['Report Date'], line)
                if report_date:
                    if report_date.group(1):
                        self.extracted_data['Report Date'] = datetime.strptime(report_date.group(1),"%Y.%b.%d")
                    else:
                        self.extracted_data['Report Date'] = datetime.strptime(report_date.group(2),"%d-%b-%Y")
                    continue

                air = re.match(pattern['Air'], line)
                if air:
                    self.extracted_data['Air'] = float(air.group(1))
                    self.extracted_data['Specified Air'] = [float(air.group(2))-float(air.group(3)), float(air.group(2))+float(air.group(3))]
                    continue

                if line.lower().strip().startswith('admixtures (ml/m'):
                    for j in range(i, len(self.data.split('\n'))):
                        if self.data.split('\n')[j].lower().strip().startswith('curing conditions'):
                            self.extracted_data['Admixtures'] = self.data.split('\n')[i+1:j]
                            break
                    continue

                if line.lower().strip() == "location":
                    for j in range(i, len(self.data.split('\n'))):
                        if self.data.split('\n')[j].lower().strip().startswith('supplier'):
                            self.extracted_data['Location Comments'] = self.data.split('\n')[i+1:j]
                            break
                    continue

                if line.lower().strip() == "comments":
                    for j in range(i, len(self.data.split('\n'))):
                        if self.data.split('\n')[j].lower().strip().startswith('load'):
                            self.extracted_data['Other Comments'] = self.data.split('\n')[i+1:j]
                            break
                    continue
                
                slump = re.match(pattern['Slump'], line)
                if slump:
                    self.extracted_data['Slump'] = int(slump.group(1))
                    self.extracted_data['Specified Slump'] = [int(slump.group(2))-int(slump.group(3)),int(slump.group(2))+int(slump.group(3))]
                    continue

                mix = re.match(pattern['Mix'], line)
                if mix:
                    self.extracted_data['Mix Number'] = mix.group(1)
                    continue

                load = re.match(pattern['Load Volume'], line.strip())
                if load:
                    self.extracted_data['Load Vol'] = float(load.group(1))

        except Exception as e:
            self.log_error('z',str(e))

    def Kontur(self):
        try:
            pattern = {
                "Test Data": r"([A-Z]) *Cylinder *(\d*) *(Lab|Field) *((\d{2}-\w{3})|(\w{3}.\d{2})-? *(\d*) *[A-Za-z ]*)?\d{3}.\d *\d{3}.\d( *\d* *(\d*.\d))?",
                "Specified Strength": r"SPECIFIED *STRENGTH: *(\d*) *MPa *@ *(\d*) * DAYS",
                "Set Data": r"SET *NO.:(\d*) *SPECIMENS: *(\d*) *CAST: *(\d{4}.\w{3}.\d{2}) *TRANSPORTED: *(\d{4}.\w{3}.\d{2})",
                "Report Date": r".*Page *1 *of *\d* *(\d*.\w{3}.\d*)|(\d*-\w{3}-\d{4})",
                "Air": r".*AIR: *(\d{1,2}.?\d?) *% *SPEC.: *(\d{1,2}.?\d?) *± *(\d{1,2}.?\d?)",
                "Slump": r".*SLUMP: *(\d*) *mm *SPEC.: *(\d*) *± *(\d*)",
                "Mix": r".*MIX *NO.:? *(.*)$",
                "Load Volume": r"^LOAD *VOL.: *(\d*) m3"
            }
            for i, line in enumerate(self.data.split('\n')):
                specified_strength = re.match(pattern['Specified Strength'], line.strip())
                if specified_strength:
                    self.extracted_data["Specified Strength"] = int(specified_strength.group(1))
                    self.extracted_data["Specified Strength Days"] = int(specified_strength.group(2))

                set_data = re.match(pattern["Set Data"], line)
                if set_data:
                    self.extracted_data["Set Num"] = set_data.group(1)
                    self.extracted_data["Specimens"] = int(set_data.group(2))
                    self.extracted_data["Cast Date"] = datetime.strptime(set_data.group(3),"%Y.%b.%d")
                    self.extracted_data["Transported Date"] = datetime.strptime(set_data.group(4),"%Y.%b.%d")
                    continue

                test_data = re.match(pattern["Test Data"], line)
                if test_data:
                    data = [test_data.group(1), test_data.group(3), test_data.group(6), test_data.group(7), test_data.group(9)]
                    if data[2]:
                        if datetime.strptime(data[2] + "-" +str(datetime.now().year), "%b.%d-%Y") > datetime.now():
                            data[2] = datetime.strptime(data[2] + "-" +str(datetime.now().year - 1), "%b.%d-%Y")
                        else:
                            data[2] = datetime.strptime(data[2] + "-" +str(datetime.now().year), "%b.%d-%Y")
                    df = pd.DataFrame([data],columns = self.test_data_cols)
                    self.extracted_data['Test Data'] = pd.concat([self.extracted_data['Test Data'], df], ignore_index=True)
                    continue
                
                report_date = re.match(pattern['Report Date'], line)
                if report_date:
                    if report_date.group(1):
                        self.extracted_data['Report Date'] = datetime.strptime(report_date.group(1),"%Y.%b.%d")
                    else:
                        self.extracted_data['Report Date'] = datetime.strptime(report_date.group(2),"%d-%b-%Y")
                    continue

                air = re.match(pattern['Air'], line)
                if air:
                    self.extracted_data['Air'] = float(air.group(1))
                    self.extracted_data['Specified Air'] = [float(air.group(2))-float(air.group(3)), float(air.group(2))+float(air.group(3))]
                    continue

                if line.lower().strip().startswith('admixtures'):
                    for j in range(i, len(self.data.split('\n'))):
                        if self.data.split('\n')[j].lower().strip().startswith('curing'):
                            self.extracted_data['Admixtures'] = self.data.split('\n')[i+1:j]
                            break
                    continue

                if line.lower().strip() == "location:":
                    for j in range(i, len(self.data.split('\n'))):
                        if self.data.split('\n')[j].lower().strip().startswith('supplier'):
                            self.extracted_data['Location Comments'] = self.data.split('\n')[i+1:j]
                            break
                    continue

                if line.lower().strip() == "comments:":
                    for j in range(i, len(self.data.split('\n'))):
                        if self.data.split('\n')[j].lower().strip().startswith('load'):
                            self.extracted_data['Other Comments'] = self.data.split('\n')[i+1:j]
                            break
                    continue
                
                slump = re.match(pattern['Slump'], line)
                if slump:
                    self.extracted_data['Slump'] = int(slump.group(1))
                    try:
                        self.extracted_data['Specified Slump'] = [int(slump.group(2))-int(slump.group(3)),int(slump.group(2))+int(slump.group(3))]
                    except:
                        self.extracted_data['Specified Slump'] = []
                    continue

                mix = re.match(pattern['Mix'], line)
                if mix:
                    self.extracted_data['Mix Number'] = mix.group(1)
                    continue

                load = re.match(pattern['Load Volume'], line.strip())
                if load:
                    try:
                        self.extracted_data['Load Vol'] = float(load.group(1))
                    except:
                        self.log_error('e', 'Missing information on Load volume')

        except Exception as e:
            self.log_error('z',str(e))

    def check_errors(self):
        self.err = {
            'a':'Template for this company is not defined',
            'b': 'Specimen # in extracted PDF does not match scrapped test data from PDF',
            'c': 'Slump issue',
            'd': 'Air % Issue',
            'e': 'Missing Information',
            'z': 'Some other error'
        }
        data = self.extracted_data
        if len(data['Test Data']) != data['Specimens']:
            self.log_error('b', f"Specimens: {data['Specimens']}, Avail. Data: {len(data['Test Data'])}")

        if data['Air'] and data['Specified Air'] != []:
            if data['Air'] > data['Specified Air'][1]:
                self.log_error('d', f"Air: {data['Air']}% > Allow. max:{data['Specified Air'][1]}%")
            elif data['Air'] < data['Specified Air'][0]:
                self.log_error('d', f"Air: {data['Air']}% < Allow. min:{data['Specified Air'][0]}%")
        else:
            self.log_error('e', 'No Air content data found.')

        if data['Slump'] and data['Specified Slump'] != []:
            if data['Slump'] > data['Specified Slump'][1]:
                self.log_error('c', f"Slump: {data['Slump']}mm > Allow. max:{data['Specified Slump'][1]}mm")
            elif data['Slump'] < data['Specified Slump'][0]:
                self.log_error('c', f"Slump: {data['Slump']}mm <  Allow. min:{data['Specified Slump'][0]}mm")
        else:
            self.log_error('e', 'Slump info not found.')
        self.extracted_data = data

In [ ]:
class xlsx():
    def __init__(self, file):
        self.wb = openpyxl.Workbook()
        self.wb.active.title = "Summary"
        self.file = file
        self.saved = False
        self.define_styles()

    def define_styles(self):
        self.fonts = {
            'h1': Font(name='Arial', size=12, italic=False, bold=True, color='FF000000', strike=False,underline='none',vertAlign=None)
        }

        self.alignments = {
            'center':Alignment(horizontal='center', vertical='bottom', text_rotation=0, wrap_text=False, shrink_to_fit=False, indent=0)
        }

        self.borderthickness = {
            'thin': Side(border_style="thin", color="000000"),
            'double': Side(border_style="double", color="000000"),
            'thick': Side(border_style="thick", color="000000")
        }
    def write_excel_data(self, ws, data, start_row, start_col = 1):
        """Writes a array of array to excel worksheet

        Args:
            ws (Openpyxl Worksheet Class): Sheet to which the data is written
            data (Array of Arrays): Data to write in array of array format; [[R1C1, R1C2],[R2C1, R2C2]]
            start_row (int): Start row; 1-indexed;Excel row to write to
        """
        #print(f"Writing {len(data)} rows of data to {ws}, starting at {start_row}")
        for i in range(start_row, start_row + len(data)):
            for j in range(start_col + len(data[i-start_row]) - 1):                
                cell = (openpyxl.utils.get_column_letter(1+j)) + str(i)
                ws[cell] = data[i-start_row][j]
                self.saved = False
        
        return start_row + len(data)

    def draw_border(self, ws, data, start_row, start_col=1):
        max_rows = len(data) + start_row
        max_cols = start_col

        for row in data:
            max_cols = max(max_cols, len(row)+start_col)

        for i in range(start_row, max_rows):
            for j in range(start_col-1, max_cols-1):
                cell = openpyxl.utils.get_column_letter(j+1) + str(i)
                left = self.borderthickness['thin']
                right = self.borderthickness['thin']
                top = self.borderthickness['thin']
                bottom = self.borderthickness['thin']
                if i == start_row:
                    top = self.borderthickness['thick']
                if i == max_rows-1:
                    bottom = self.borderthickness['thick']
                if j == start_col-1:
                    left = self.borderthickness['thick']
                if j == max_cols-2:
                    right = self.borderthickness['thick']
                
                ws[cell].border = Border(left=left, right=right, top=top, bottom=bottom)
                ws[cell].alignment = Alignment(wrap_text=True, vertical="center")

    def create_data_sheet(self, title, sample_data):
        ws = self.wb.create_sheet(index=0, title=title)
        self.saved = False
        data = [
            ['Powell River WWTP'],
            ['Concrete Compressive Strength Summary'],
            [f"{sample_data['Mix Number']}: {sample_data['Specified Strength']}MPa @ {sample_data['Specified Strength Days']} days"],
            ['']
        ]
        next_row = self.write_excel_data(ws, data, 1)
        data = [
            ['Set', 'Cylinder', 'Description', 'Date', 'Date', 'Date', 'Age', 'Test', r'% of', 'Air', 'Slump', 'Comments'],
            ['#', 'ID', '', 'Cast', 'Received', 'Tested', 'Days', 'MPa', 'Design', '%', 'mm', '']
        ]
        self.draw_border(ws, data,next_row)
        next_row = self.write_excel_data(ws, data, next_row)

        ws.column_dimensions["C"].width = 30
        ws.column_dimensions["D"].width = 15
        ws.column_dimensions["E"].width = 15
        ws.column_dimensions["F"].width = 15
        ws.column_dimensions["L"].width = 30
        
        for cells in ws["1:3"]:
            for cell in cells:
                cell.font = self.fonts['h1']
        for cells in ws["4:6"]:
            for cell in cells:
                cell.alignment = self.alignments['center']

        ws.freeze_panes = 'A7'
        return ws, next_row

    def write_set_data(self, ws, data, startrow,startcol=1):
        data_to_write = []

        for index, row in data['Test Data'].iterrows():
            if index == 0:
                location_comment = [x.strip().title() for x in data['Location Comments']]
                other_comment = [x.strip().title() for x in data['Other Comments']]
                data_to_write.append([
                    data['Set Num'], 
                    row['Specimen'], 
                    '\n'.join(location_comment),
                    datetime.strftime(data['Cast Date'], "%d-%B-%Y"),
                    datetime.strftime(data['Transported Date'], "%d-%B-%Y"),
                    datetime.strftime(row['Test_Date'], "%d-%B-%Y"),
                    int(row['Age']) if row['Age'] != None else None,
                    float(row['Compressive_Str']) if row['Compressive_Str'] != None else None,
                    round(float(row['Compressive_Str']) / data['Specified Strength'],2) if row['Compressive_Str'] != None else None,
                    data['Air'],
                    data['Slump'],
                    '\n'.join(other_comment)
                    ])
            else:
                data_to_write.append([
                    '', '', '', '', '', '',
                    int(row['Age']) if row['Age'] != None else None,
                    float(row['Compressive_Str']) if row['Compressive_Str'] != None else None,
                    round(float(row['Compressive_Str']) / data['Specified Strength'],2) if row['Compressive_Str'] != None else None,
                ])


        next_row = self.write_excel_data(ws, data_to_write, startrow)
        self.draw_border(ws, data_to_write, startrow)
        return next_row

    def plot_sheet(self, sets, filename):
        x_axis = []
        req =[]
        str_days = []
        for set_key in sorted(sets.keys()):
            for index, row in sets[set_key].extracted_data['Test Data'].iterrows():
                if row['Compressive_Str'] and int(row['Age']) <= sets[set_key].extracted_data['Specified Strength Days']: 
                        if not int(row['Age']) in str_days:
                            str_days.append(int(row['Age']))

        df = pd.DataFrame(columns=sorted(str_days))

        for set_key in sorted(sets.keys()):
            #Create dataframe with setwise data      
            df.loc[set_key] = 0  
            for index, row in sets[set_key].extracted_data['Test Data'].iterrows():
                if row['Compressive_Str'] and int(row['Age']) <= sets[set_key].extracted_data['Specified Strength Days']:
                        df[int(row['Age'])][set_key] = float(row['Compressive_Str'])

            str_days = sorted(str_days)
            x_axis.append(str(set_key))
            req.append(sets[set_key].extracted_data['Specified Strength'])

        plt.clf()
        plt.bar(x_axis, req, color='r', label='Target Str.')
        plt.axhline(y=req[0],linewidth=1, color='r', linestyle='--')
        c = np.arange(1, len(str_days) + 1)
        norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
        cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Blues)
        cmap.set_array([])
        i=0
        for index, row in df.transpose().iterrows():
            plt.bar(x_axis, row.to_list(), width=0.8, color=cmap.to_rgba(i + 1), alpha=1, label=f"{row.name} days")
            i+=1
        plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        plt.xlabel('Set #')
        plt.ylabel('MPa')
        plt.title("How to read: You shouldn't see any red bars in graph", fontsize=8)
        
        target={'str':sets[set_key].extracted_data['Specified Strength'], 'day': sets[set_key].extracted_data['Specified Strength Days']}
        plt.suptitle(f"{sets[set_key].extracted_data['Mix Number']}: {target['str']}MPa @ {target['day']} days test summary")
        plt.savefig(filename,bbox_inches='tight')
        return filename
        #plt.plot()

    def save(self):
        self.wb.save(self.file)
        self.saved = True


In [ ]:
class PDF(FPDF):
    def header(self):
        # Logo
        self.image(r"img_ae_logo_c_230.png", 10, 8, 33)
        # Arial bold 15
        self.set_font('Times', 'B', 20)
        # Move to the right
        self.cell(80)
        # Title
        self.cell(30, 10, 'Concrete Test Report Summary', 0, 0, 'C')
        self.cell(80)
        self.set_font('Times', '', 11)
        self.cell(0, 4, 'Arun Kishore', 0, 1, 'R')
        self.cell(0, 4, datetime.now().strftime('%B %d, %Y'), 0, 0, 'R')
        # Line break
        self.ln(20)

    # Page footer
    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Times', 'I', 8)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

    def draw_table(self, header, data, cell_width, heading='', subheading=''):
        """Draws a table in PDF based on provided input

        Args:
            pdf (<FPDF class>): _description_
            header (tuple): A tuple with header row values
            data (Array of Arrays): Array with table data
            cell_width (Array): Array of int with col widths
            heading (str, optional): Table Title. Defaults to ''.
            subheading (str, optional): Table subtitle. Defaults to ''.
        """
        #print(header)
        #print(data)
        effective_page_width = self.w - 2*self.l_margin
        cell_offset = (effective_page_width - sum(cell_width))/2

        if heading != '':
            self.set_font('Times', 'B', 14)
            self.cell(w=0, h=5, txt=heading, ln=1, align='C')

        if subheading != '':
            self.set_font('Times', 'I', 10)
            self.cell(w=0, h=4, txt=subheading, ln=1, align='C')

        if cell_offset > 0:
            self.cell(cell_offset)

        self.set_font('Times', 'B', 10)
        for i, datum in enumerate(header):
            self.cell(cell_width[i], self.font_size * 1.5, datum, border=1,align='C')
        self.ln(self.font_size * 1.5)

        self.set_font('Times', '', 10)
        for row in data:
            if cell_offset > 0:
                self.cell(cell_offset)
            for i, datum in enumerate(row):
                self.cell(cell_width[i], self.font_size * 1.5, str(datum), border=1,align='C')
            self.ln(self.font_size * 1.5)
        self.ln(self.font_size * 1.5)
    


### Process PDF Files

In [ ]:
parsed_data = []
combine = {
    'W-1': ['W-1 180', 'W1-150', 'W-1 (200)', 'W-1(32MPA)', 'W1'],
    'W-2': ['W2', 'W-2-PEA'],
    'W-3': ['W3'],
    'W-4': ['W4'],
    'W-5': ['W5'],
    'W-6': ['W6', 'W-6 PEA', 'W-6(56)', 'W-6(32MPA)'],
    'W-7': ['W-72']
}

for file in tqdm(files, desc="Prcessing PDFs",unit="files", colour="green",):
    data = concrete_test(file)
    data.combine_sheets(combine)
    parsed_data.append(data)

sheets = {}
for item in parsed_data:
    if not item.extracted_data['Mix Number'] in sheets.keys():
        sheets[item.extracted_data['Mix Number']] = item

print("Following are the list of sheets that will be currently produced")
print(list(sheets.keys()))


## User Input Needed - Combine sheets

In [ ]:
combine = {
    'W-1': ['W-1 180', 'W1-150', 'W-1 (200)', 'W-1(32MPA)', 'W1'],
    'W-2': ['W2', 'W-2-PEA', 'W2 (165)'],
    'W-3': ['W3'],
    'W-4': ['W4'],
    'W-5': ['W5'],
    'W-6': ['W6', 'W-6 PEA', 'W-6(56)', 'W-6(32MPA)'],
    'W-7': ['W-72']
}
for file in parsed_data:
    file.combine_sheets(combine)

### User Input Needed - Define the strength requirements

In [ ]:
#Compile a list of sheets
sheets = {}
for item in parsed_data:
    if not item.extracted_data['Mix Number'] in sheets.keys():
        if item.extracted_data['Mix Number']:
            sheets[item.extracted_data['Mix Number']] = ""

specified = {
    'W-1': [35,56, 'Watertight Concrete'],
    'W-2': [35, 28, 'Exterior Structural Concrete'],
    'W-3': [35, 28, 'Interior Structural Concrete'],
    'W-4': [32, 56, 'Large Manhole Outfall'],
    'W-5': [15, 56, 'Lean Concrete'],
    'W-6': [32, 56, 'Bioreactor tank base slab'],
    'W-7': [25, 56, 'Manhole and Footings'],
    '11 BLOCK F': [25, 28, 'Masonry Mix']
}

#### Report Generation

### Create Excel

In [ ]:
#Update the sample data in sheets dict to point to corrent strength item
for sheet in sheets.keys():
    for item in parsed_data:
        data = item.extracted_data
        if data['Mix Number'] == sheet:
            item.extracted_data['Specified Strength'] = specified[sheet][0]
            item.extracted_data['Specified Strength Days'] = specified[sheet][1]
            sheets[sheet] = item


xl_file  = os.path.join(folder, os.path.basename(folder) + '.xlsx')
xl = xlsx(xl_file)
for sheet in sheets.keys():
    if sheet == None:
        continue
    ws, current_row = xl.create_data_sheet(sheet, sheets[sheet].extracted_data)
    
    sets = {}

    for item in parsed_data:
        if item.extracted_data['Mix Number'] != sheet:
            continue
        if item.extracted_data['Set Num'] == None:
            continue
        set_num = int(item.extracted_data['Set Num'])
        report_date = item.extracted_data['Report Date']
        if not set_num in sets.keys():
            sets[set_num] = item
        elif len(item.extracted_data['Test Data'].dropna()) > len(sets[set_num].extracted_data['Test Data'].dropna()):
            sets[set_num] = item

    for set_key in sorted(sets.keys()):
        current_row = xl.write_set_data(ws, sets[set_key].extracted_data, current_row+1)
    
    sheets[sheet].figure = xl.plot_sheet(sets, sheet + '.png')
xl.save()
print(f"Generated excel file is saved at: {xl.file}")
print(f"{len(sheets.keys())} plot files are also generated and saved in the above folder")
#subprocess.Popen('example_copy.xlsx', shell=True)

### Create PDF

In [ ]:
pdf = PDF()
pdf.alias_nb_pages()
pdf.add_page()
effective_page_width = pdf.w - 2*pdf.l_margin

template_excluded_pdf = [x for x in parsed_data if len(x.extracted_data['Errors'][x.extracted_data['Errors']['Error Code'] == 'a']) > 0]
missing_info_pdf = [x for x in parsed_data if len(x.extracted_data['Errors'][x.extracted_data['Errors']['Error Code'] == 'e']) > 0]
other_err_pdf = [x for x in parsed_data if len(x.extracted_data['Errors'][x.extracted_data['Errors']['Error Code'] == 'e']) > 0]
slump_pdf = [x for x in parsed_data if len(x.extracted_data['Errors'][x.extracted_data['Errors']['Error Code'] == 'c']) > 0]
air_pdf = [x for x in parsed_data if len(x.extracted_data['Errors'][x.extracted_data['Errors']['Error Code'] == 'd']) > 0]
summary = f"""
The below summary is based on information from concrete test result PDFs located at: {folder}.

A total of {len(files)} PDFs were considered."""

if len(template_excluded_pdf) > 0 or len(missing_info_pdf) > 0:
    summary += f" Out of which:\n"
    if len(template_excluded_pdf) > 0:
        summary += f"\t- {len(template_excluded_pdf)} were excluded since they didn't meet the defined template criteria.\n"
    if len(missing_info_pdf) > 0:
        summary += f"\t- {len(missing_info_pdf)} were found to have some missing information.\n"""

if len(air_pdf) > 0 or len(slump_pdf) > 0:
    summary += f"\nFrom the supplied PDFs:\n"
    if len(air_pdf) > 0:
        summary += f"\t- {len(air_pdf)} were found to have air content outside of specified range.\n"
    if len(slump_pdf) > 0:
        summary += f"\t- {len(slump_pdf)} were found to have slump outside of specified range.\n"


    summary += f"Possible explanation for the above:\n"
    if len(air_pdf) > 0:
        summary += f"\t- For Air content, since we have multiple PDFs relating to a single set (7-day, 14-day,...) multiple PDFs flagged for violation might all pertain to a small # of samples.\n"
    if len(slump_pdf) > 0:
        summary += f"\t- For Slump, it is possible that the specified range is before the addition of superplasticizer but the reported range might have been after the addition.\n"
    
if len(template_excluded_pdf) > 0:
    summary += f"""\nA list of exluded files has been listed below.\nIt is possible that a few of these files were rejected due to minor formatting differences, but if a large number of false negative is observed, contact Arun to redefine the templates to account for the changes.
    """

#Summary
pdf.set_font('Times', 'B', 18)
#pdf.cell(w=0, h=10, txt='Summary:', border=0, ln=1, fill=False)
pdf.cell(w=0, h=10, txt='Summary', border=0, ln=1, fill=False, align='C')
pdf.set_font('Times', '', 12)
pdf.multi_cell(w=0, h=6, txt=summary, border=0, fill=False)
pdf.ln(pdf.font_size * 2.5)

if len(template_excluded_pdf) > 0:
    #Excluded Files
    data = []
    for file in template_excluded_pdf:
        data.append([file.extracted_data['Filename']])

    pdf.draw_table(
        header= ['Filename'],
        data = data,
        cell_width= [2*effective_page_width / 3],
        heading= 'Excluded Files',
        subheading= 'List of files excluded from analysis since template does not match definied templates'
    )

if len(missing_info_pdf) > 0:
    #Missing Information
    pdf.add_page()
    data = []
    for file in missing_info_pdf:
        issue = ""
        err_desc = file.extracted_data['Errors'][file.extracted_data['Errors']['Error Code'] == 'e']['Description'].to_list()
        if 'No Air content data found.' in err_desc:
            issue = "Air content"
        if 'Slump info not found.' in err_desc:
            if issue != "":
                issue += ", "
            issue += "Slump"
        if 'Missing information on Load volume' in err_desc:
            if issue != "":
                issue += ", "
            issue += "Load vol"
        data.append([file.extracted_data['Filename'], issue])

    pdf.draw_table(
        header= ('Filename', 'Missing Info'),
        data = data,
        cell_width= (2*effective_page_width / 3 - 20, effective_page_width / 3 - 10),
        heading= 'Files with missing information',
        subheading= 'List of files where specific information was noted to be missing'
    )

pdf.add_page()
pdf.set_font('Times', 'B', 18)
pdf.cell(w=0, h=10, txt='Target strength vs Achived strength', border=0, ln=1, fill=False, align='C')
pdf.set_font('Times', '', 12)
pdf.multi_cell(w=0, h=6, txt='The charts plot the target strengths vs achieved strengths.', border=0, fill=False, align='C')
pdf.ln(pdf.font_size * 1.5)
for sh in sheets.keys():
    pdf.cell(20)
    pdf.image(sheets[sh].figure)
    pdf.ln(pdf.font_size * 2.5)

if len(air_pdf) > 0 or len(slump_pdf) > 0:
    #Air content Exceeded
    pdf.add_page()
    air_data = []
    for file in air_pdf:
        air_data.append([file.extracted_data['Filename']])
    slump_data = []
    for file in slump_pdf:
        slump_data.append([file.extracted_data['Filename']])
        
    pdf.draw_table(
        header= ['Missing Air %', 'Missing Slump'],
        data = zip_longest(air_data, slump_data),
        cell_width= [effective_page_width / 2, effective_page_width / 2],
        heading= 'Files where Air content or slump falls outside specified range',
        subheading= 'See summary for a brief description and possible explanation for below list of files'
    )

pdf.file = os.path.join(folder, os.path.basename(folder) + '.pdf')
pdf.output(pdf.file, 'F')
print(f"PDF file can be located at: {pdf.file}")
subprocess.Popen(pdf.file, shell=True)

del pdf

### Finish

In [ ]:
for sh in sheets.keys():
    try:
        os.remove(sheets[sh].figure)
        print(f"File removed: {sheets[sh].figure}")
    except Exception as e:
        print(e)

for file in parsed_data:
    del file
parsed_data = []
print("All data flushed from memory")